### Load up selenium and the Chrome webdriver

In [4]:
import selenium
from selenium import webdriver
from selenium.common import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [5]:
driver = webdriver.Chrome()

In [6]:
driver.get('https://generalssb-prod.ec.njit.edu/BannerExtensibility/customPage/page/stuRegCrseSched')

### In the left navigation, find the 16th entry ("CS") and click on it

#### This will cause the main content area to load all of the CS courses

In [7]:
driver.implicitly_wait(5)
left_link = driver.find_element(By.ID, 'pbid-subjListTableSubjectLink-16')

In [8]:
cs_link = left_link.find_element(by=By.TAG_NAME, value='a')
cs_link.click()

In [9]:
my_span = driver.find_element(by=By.ID, value='pbid-courseListSectionDetailSections-0')

In [10]:
print(my_span)

<selenium.webdriver.remote.webelement.WebElement (session="e450cebf5039eea18dee397f56a193d4", element="f.5D5FA68BADDAF62DDB3B6DC55D9E7BA2.d.B5B1DF25EE1A5DD6D852C3FC82167527.e.51")>


In [11]:
course_headers = my_span.find_elements(by=By.TAG_NAME, value="H4")

In [12]:
course_tables = my_span.find_elements(by=By.TAG_NAME, value="table")

In [13]:
print("I have {} headers and {} tables".format(len(course_headers), len(course_tables)))

I have 88 headers and 88 tables


In [14]:
def create_column_headings(table_headers):
    loc_columns = ['Course_name']
    for i in range(len(table_headers)):
        loc_columns.append(table_headers[i].text)
    return loc_columns

In [15]:
def create_course_listings(c_headers, c_tables):
    c_list = []
    for i in range(len(c_headers)):
        course_name = c_headers[i].text
#       collect the rows for each section of the course
        t_rows = c_tables[i].find_elements(by=By.CLASS_NAME, value='success')
#       for each row, get all of the cells
        for k in range(len(t_rows)):
            course_row = []
            course_row.append(course_name)
            t_cells = t_rows[k].find_elements(by=By.TAG_NAME, value='td')
            for l in range(len(t_cells)):
                course_row.append(t_cells[l].text)
            c_list.append(course_row)
    return c_list


In [16]:
# create column headings by using the 'th' cells of the first table
t_headers = course_tables[0].find_elements(by=By.TAG_NAME, value="th")
columns = create_column_headings(t_headers)

In [17]:
# use the list of course_headers and the list of course_tables to build
#   individual rows - one for each course/section
course_list = create_course_listings(course_headers, course_tables)
df = pd.DataFrame(course_list, columns=columns)
driver.quit()

In [18]:
df

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
0,CS 100 - ROADMAP TO COMPUTING,002,11742,TR,4:00 PM - 5:20 PM,CKB 217,Open,80,71,"Spirollari, Junilda",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
1,CS 100 - ROADMAP TO COMPUTING,004,11743,MR,2:30 PM - 3:50 PM,CKB 317,Open,40,9,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
2,CS 100 - ROADMAP TO COMPUTING,006,11744,MR,8:30 AM - 9:50 AM,CKB 223,Open,40,14,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
3,CS 100 - ROADMAP TO COMPUTING,008,11745,MR,10:00 AM - 11:20 AM,CKB 317,Open,40,28,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
4,CS 100 - ROADMAP TO COMPUTING,010,11746,MR,1:00 PM - 2:20 PM,CKB 330,Open,40,19,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416,CS 792 - PRE-DOCTORAL RESEARCH,623,12221,,TBA,,Open,19,0,"Theodoratos, Dimitrios",Face-to-Face,3,Book,See department for permission to enroll.
417,CS 792 - PRE-DOCTORAL RESEARCH,624,12222,,TBA,,Open,19,1,"Wang, Guiling",Face-to-Face,3,Book,See department for permission to enroll.
418,CS 792 - PRE-DOCTORAL RESEARCH,625,12223,,TBA,,Open,19,0,"Mili, Ali",Face-to-Face,3,Book,See department for permission to enroll.
419,CS 792 - PRE-DOCTORAL RESEARCH,626,12224,,TBA,,Open,19,0,"Chakareski, Jakov",Face-to-Face,3,Book,See department for permission to enroll.


In [20]:
df.describe()

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
count,422,422,422,422,422,422,422,422,422,422,422,422,422,422
unique,73,69,422,16,17,45,1,14,44,87,4,3,1,15
top,CS 488 - INDEPENDENT STUDY IN CS,002,11742,,TBA,,Open,19,0,"Wu, Chase",Face-to-Face,3,Book,See department for permission to enroll.
freq,48,28,1,327,327,333,422,319,281,11,414,377,422,317


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_name    422 non-null    object
 1   Section        422 non-null    object
 2   CRN            422 non-null    object
 3   Days           422 non-null    object
 4   Times          422 non-null    object
 5   Location       422 non-null    object
 6   Status         422 non-null    object
 7   Max            422 non-null    object
 8   Now            422 non-null    object
 9   Instructor     422 non-null    object
 10  Delivery Mode  422 non-null    object
 11  Credits        422 non-null    object
 12  Info           422 non-null    object
 13  Comments       422 non-null    object
dtypes: object(14)
memory usage: 46.3+ KB


In [22]:
df['Max'] = df['Max'].astype('int')
df['Now'] = df['Now'].astype('int')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Course_name    422 non-null    object
 1   Section        422 non-null    object
 2   CRN            422 non-null    object
 3   Days           422 non-null    object
 4   Times          422 non-null    object
 5   Location       422 non-null    object
 6   Status         422 non-null    object
 7   Max            422 non-null    int64 
 8   Now            422 non-null    int64 
 9   Instructor     422 non-null    object
 10  Delivery Mode  422 non-null    object
 11  Credits        422 non-null    object
 12  Info           422 non-null    object
 13  Comments       422 non-null    object
dtypes: int64(2), object(12)
memory usage: 46.3+ KB


In [24]:
df = df[~df['Course_name'].str.contains('CS 488')]
df = df[~df['Course_name'].str.contains('CS 700B')]
df = df[~df['Course_name'].str.contains('CS 701B')]
df = df[~df['Course_name'].str.contains('CS 725')]
df = df[~df['Course_name'].str.contains('CS 792')]
df = df[~df['Course_name'].str.contains('CS 790A')]
df = df[~df['Course_name'].str.contains('CS 726')]

df

,Course_name,Section,CRN,Days,Times,Location,Status,Max,Now,Instructor,Delivery Mode,Credits,Info,Comments
0,CS 100 - ROADMAP TO COMPUTING,002,11742,TR,4:00 PM - 5:20 PM,CKB 217,Open,80,70,"Spirollari, Junilda",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
1,CS 100 - ROADMAP TO COMPUTING,004,11743,MR,2:30 PM - 3:50 PM,CKB 317,Open,40,9,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
2,CS 100 - ROADMAP TO COMPUTING,006,11744,MR,8:30 AM - 9:50 AM,CKB 223,Open,40,14,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
3,CS 100 - ROADMAP TO COMPUTING,008,11745,MR,10:00 AM - 11:20 AM,CKB 317,Open,40,27,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
4,CS 100 - ROADMAP TO COMPUTING,010,11746,MR,1:00 PM - 2:20 PM,CKB 330,Open,40,17,"Nemane, Nikita",Face-to-Face,3,Book,This course has common exams.\nSee https://www...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327,CS 785 - ST: ADVANCED PROGRAMMING LANGUAGES,002,12133,T\nF,10:00 AM - 11:20 AM\n10:00 AM - 11:20 AM,CKB 220\nCULM 110,Open,25,2,"Neamtiu, Iulian",Face-to-Face,3,Book,
328,CS 785 - ST: ALGORITHMS FOR ENABLING RESPONSIB...,006,12135,TR,8:30 AM - 9:50 AM,CKB 114,Open,20,3,"Basu Roy, Senjuti",Face-to-Face,3,Book,
329,CS 785 - ST: DATABASE SECURITY,004,12134,MW,10:00 AM - 11:20 AM,TIER 111,Open,20,6,"Sharma, Shantanu",Face-to-Face,3,Book,
330,CS 786 - ST: DIMENSIONALITY AND SCALABILITY IN AI,854,16298,R,6:00 PM - 8:50 PM,,Open,20,11,"Houle, Michael",Synchronous Online,3,Book,Synchronous Online Course


In [25]:
df[['Instructor','Max', 'Now']].groupby(['Instructor']).sum().tail(50)


,Max,Now
Instructor,,
"Borcea, Cristian",30,8
"Calvin, James",100,73
"Eljabiri, Osama",80,58
"Eren, Canan",45,12
"Eskandari, Marzieh",43,6
"Estes, Michael",120,29
"Gerbessiotis, Alexandros",120,10
"Hamidli, Fuad",141,77
"Hoover, Amy",73,6
